Testing the robustness of LSTM model for the paper "Confused or not Confused" (https://github.com/nateanl/EEG_Classification)

Just for loop the thing without seeding random generator and print average accuracy among other metrics


In [1]:
import csv
import numpy as np
import tensorflow as tf
#np.random.seed(1337)  # for reproducibility
from sklearn import svm
from sklearn import cross_validation
from sklearn import tree
from sklearn import neighbors, datasets
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import scipy.io as sio
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers import BatchNormalization
from keras.layers.core import Dense
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score 


from keras.layers.wrappers import Bidirectional
log= open('5_fold_log_1125.txt','w')
features= []
label = []
feature = []
with open('./EEG_data.csv','r') as f:
    reader = csv.reader(f)
    i=0
    for row in reader:
        if(i==0):
            i+=1
            print(row)
            continue
        for i in range(0,len(row)):
            row[i] = float(row[i])
        features.append(row[0:14])
        label.append(row[14])

X = np.asarray(features)
Y = np.asarray(label)
features = {}
output = {}
print(X.shape[1])
for i in range(X.shape[0]):
    tu = int(X[i][0]*10 + X[i][1])
    if tu not in list(features.keys()):
        features[tu] = X[i][2:14]
    elif features[tu].shape[0]<1344:
        features[tu] = np.concatenate((features[tu],X[i][2:14]),axis =0)
    output[tu]= Y[i]

input = np.zeros((100,1344),dtype = float)
labels = np.zeros((100,1),dtype = int)
for i in list(features.keys()):
    input[i,:] = features[i]
    labels[i] = output[i]

accs = []
f1s = []
roc_aucs = []
for a in range(10):
    print('\nCross-validate LSTM model, iteration', a, '\n')
    accuracy = 0.0
    for i in range(0,5):
        X_train, X_test, Y_train, Y_test = train_test_split(input, labels, test_size=0.2)
        X_train = X_train.reshape(80,112,12)
        X_test = X_test.reshape(20,112,12)
        y_train = np.zeros((80,112),dtype='int')
        y_test = np.zeros((20,112),dtype='int')
        y_train = np.repeat(Y_train,112, axis=1)
        y_test = np.repeat(Y_test,112, axis=1)
        # create the model
        model = Sequential()
        batch_size = 20
        #model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length, dropout=0.2))
        #model.add(Dropout(0.2))
        model.add(BatchNormalization(input_shape=(112,12), mode=0, axis=2))
        model.add(Bidirectional(LSTM(50, return_sequences=False, input_shape=(112,12)), merge_mode='ave'))
        #model.add(Dropout(0.2))
        #model.add(LSTM(200, return_sequences = False, input_length=1024))
        model.add(Dense(112, activation='hard_sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['binary_accuracy'])
        #print(model.summary())
        model.fit(X_train, y_train, epochs=25, verbose=0)
        # Final evaluation of the model
        scores = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
        
        y_pred = model.predict(X_test)
        roc_auc = roc_auc_score(y_test.flatten(), np.round(y_pred.flatten()))
        f1 = f1_score(y_test.flatten(), np.round(y_pred.flatten()))
        
        print('Accuracy: {:.0f}%, F1: {:.3f}, ROC-AUC: {:.3f}'.format(scores[1]*100, f1, roc_auc))
        #log.write(str(scores[1])+'\n')
        accuracy += scores[1]
        
        accs.append(scores[1])
        f1s.append(f1)
        roc_aucs.append(roc_auc)
    print('\nAvg acc for current iteration =', accuracy/5)
    print('Current avg acc for all iterations =', np.mean(accs), '\n')

print('\n\n')
for  metric, scores in {'accuracy':accs, 'F1':f1s, 'ROC-AUC':roc_aucs}.items():
    print('Average {} for cross-validations: {:.3f}'.format(metric, np.mean(scores))) 
log.close() # you can omit in most cases as the destructor will call it

#average accuracy: 0.690000013262

/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


['SubjectID', 'VideoID', 'Attention', 'Mediation', 'Raw', 'Delta', 'Theta', 'Alpha1', 'Alpha2', 'Beta1', 'Beta2', 'Gamma1', 'Gamma2', 'predefinedlabel', 'user-definedlabeln']
14

Cross-validate LSTM model, iteration 0 



/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(input_shape=(112, 12), axis=2)`


Accuracy: 52%, F1: 0.508, ROC-AUC: 0.517
Accuracy: 59%, F1: 0.681, ROC-AUC: 0.564
Accuracy: 75%, F1: 0.705, ROC-AUC: 0.820
Accuracy: 70%, F1: 0.720, ROC-AUC: 0.719
Accuracy: 74%, F1: 0.745, ROC-AUC: 0.751

Avg acc for current iteration = 0.6601785540580749
Current avg acc for all iterations = 0.6601785540580749 


Cross-validate LSTM model, iteration 1 

Accuracy: 67%, F1: 0.681, ROC-AUC: 0.670
Accuracy: 69%, F1: 0.658, ROC-AUC: 0.686
Accuracy: 47%, F1: 0.498, ROC-AUC: 0.501
Accuracy: 56%, F1: 0.532, ROC-AUC: 0.571
Accuracy: 66%, F1: 0.700, ROC-AUC: 0.681

Avg acc for current iteration = 0.6091071128845215
Current avg acc for all iterations = 0.6346428334712982 


Cross-validate LSTM model, iteration 2 

Accuracy: 72%, F1: 0.695, ROC-AUC: 0.720
Accuracy: 45%, F1: 0.482, ROC-AUC: 0.464
Accuracy: 70%, F1: 0.695, ROC-AUC: 0.706
Accuracy: 70%, F1: 0.727, ROC-AUC: 0.708
Accuracy: 66%, F1: 0.597, ROC-AUC: 0.653

Avg acc for current iteration = 0.6454464435577393
Current avg acc for all itera

The commented average accuracy: 0.690000013262 is from the original code. It doesn't match our results, nor the ones presented in the paper (73.3%)